In [ ]:
import cv2
import numpy as np
import threading
import socket
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# --- Load YOLO ---
model = YOLO("model.pt")

# --- DeepSORT ---
# Explicitly specify the model for feature extraction to ensure consistency
deepsort_model = 'mobilenet'
tracker = DeepSort(max_age=10, max_iou_distance=0.4, nn_budget=100, embedder='mobilenet', n_init=3)

# --- Camera ---
ws, hs = 1280, 720
cap = cv2.VideoCapture(1)
cap.set(3, ws)
cap.set(4, hs)

# --- Socket TCP ---
HOST = '127.0.0.1'
PORT = 65432
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    server_socket.connect((HOST, PORT))
except ConnectionRefusedError:
    print(f"Error: Could not connect to the slave at {HOST}:{PORT}. Make sure Slave.py is running.")
    exit()

def send_data(data):
    try:
        serialized_data = data.tobytes()
        server_socket.sendall(serialized_data)
    except BrokenPipeError:
        print("Error: Connection to slave lost.")
        exit()

# --- Dữ liệu chia sẻ giữa các luồng ---
detections = []
latest_frame = None
yolo_lock = threading.Lock()

# --- Hàm YOLO chạy song song ---
def run_yolo_loop():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame_copy = latest_frame.copy()
            results = model(frame_copy)
            new_detections = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0].item()
                class_id = int(box.cls[0].item())
                if class_id == 0:  # Người
                    new_detections.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_id))
            detections = new_detections

threading.Thread(target=run_yolo_loop, daemon=True).start()

# --- Mouse callback chọn người ---
cv2.namedWindow("Master")
selected_id = None

def select_object(event, x, y, flags, param):
    global selected_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for track in param:
            if track.is_confirmed():
                x1, y1, x2, y2 = map(int, track.to_ltrb())
                if x1 <= x <= x2 and y1 <= y <= y2:
                    selected_id = track.track_id
                    # Gửi đặc trưng ngay khi chọn nếu có
                    # Trong Master.py, trong hàm select_object:
                    if hasattr(track, "features") and track.features:
                        feature = track.features[-1]
                        print(f"Master - Extracted feature shape: {feature.shape}") # Thêm dòng này
                        print(f"Master sending feature with shape: {feature.shape}")
                        send_data(feature)
                    print(f"🔍 Chọn đối tượng ID: {selected_id}")
                    break  # Chỉ chọn một đối tượng tại một thời điểm

cv2.setMouseCallback("Master", select_object, param=[])

# --- Vòng lặp chính ---
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    with yolo_lock:
        latest_frame = frame.copy()

    tracks = tracker.update_tracks(detections, frame=frame)
    cv2.setMouseCallback("Master", select_object, param=tracks)

    # Hiển thị bounding box và nhận diện đối tượng
    for track in tracks:
        if track.is_confirmed():
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            color = (0, 255, 0) if track.track_id == selected_id else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID {track.track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Master", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
server_socket.close()


0: 480x640 1 person, 235.1ms
Speed: 3.7ms preprocess, 235.1ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 284.0ms
Speed: 4.2ms preprocess, 284.0ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 211.5ms
Speed: 1.7ms preprocess, 211.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 244.1ms
Speed: 2.4ms preprocess, 244.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 251.1ms
Speed: 2.8ms preprocess, 251.1ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 245.0ms
Speed: 2.2ms preprocess, 245.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 267.7ms
Speed: 1.6ms preprocess, 267.7ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 235.8ms
Speed: 2.3ms preprocess, 235.8ms inference, 2.1ms postprocess per image at

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deep_sort import nn_matching
from deep_sort_realtime.deepsort_tracker import DeepSort

# --- Load YOLO face model ---
model = YOLO("model.pt")  # model đã được train để detect khuôn mặt



# --- Load DeepSort Tracker ---
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    max_cosine_distance=0.4,
    nn_budget=None,
    embedder="mobilenet",
    half=True,
    bgr=True,
    embedder_gpu=True,
)

# --- Load database đặc trưng khuôn mặt ---
def build_feature_database(data_folder='data'):
    features_db = {}
    for person_name in os.listdir(data_folder):
        person_path = os.path.join(data_folder, person_name)
        if not os.path.isdir(person_path):
            continue

        embeddings = []
        for file in os.listdir(person_path):
            if not file.lower().endswith(".jpg"):
                continue
            img_path = os.path.join(person_path, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            try:
                feature = tracker  # Sử dụng embedder để trích xuất đặc trưng
                embeddings.append(feature)
            except Exception as e:
                print(f"❌ Lỗi trích đặc trưng: {img_path} - {e}")
        if embeddings:
            avg_feature = np.mean(embeddings, axis=0)
            features_db[person_name] = avg_feature
    print(f"✅ Đã tải {len(features_db)} người trong database.")
    return features_db

# --- Tính độ tương đồng cosine ---
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def recognize_face(feature, features_db, threshold=0.5):
    best_match, best_score = None, -1
    for name, db_feature in features_db.items():
        score = cosine_similarity(feature, db_feature)
        if score > best_score and score > threshold:
            best_match = name
            best_score = score
    return best_match, best_score

# --- Load đặc trưng khuôn mặt ---
features_db = build_feature_database("data")

# --- Mở camera ---
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Lưu thông tin tên đã nhận diện cho mỗi track_id
track_id_to_name = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    detections = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        if cls == 0:  # class 0 là khuôn mặt
            detections.append(([x1, y1, x2, y2], conf, 'face'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        # Nếu track_id chưa có trong từ điển => nhận diện
        if track_id not in track_id_to_name:
            face_crop = frame[y1:y2, x1:x2]
            try:
                feature = tracker.embedder.get_features(face_crop)[0]  # Trích đặc trưng khuôn mặt
                name, score = recognize_face(feature, features_db, threshold=0.5)
                if name:
                    track_id_to_name[track_id] = f"{name} ({score:.2f})"
                else:
                    track_id_to_name[track_id] = "Unknown"
            except Exception as e:
                print(f"Lỗi nhận diện: {e}")
                track_id_to_name[track_id] = "Error"

        # Hiển thị thông tin
        label = track_id_to_name.get(track_id, "Processing...")
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID:{track_id} {label}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("Face Tracking & Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# --- Hàm tiền xử lý ảnh khuôn mặt ---
def preprocess_face(face_img, target_size=(128, 128)):
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = cv2.resize(face_img, target_size)
    face_img = face_img.astype(np.float32) / 255.0  # Chuẩn hóa
    return face_img

# --- Hàm tính cosine similarity ---
def cosine_similarity(a, b):
    a = a.flatten()
    b = b.flatten()
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# --- Hàm nhận diện khuôn mặt ---
def recognize_face(feature, features_db, threshold=0.6):
    best_match = ("Unknown", 0)
    for name, db_feature in features_db.items():
        similarity = cosine_similarity(feature, db_feature)
        if similarity > threshold and similarity > best_match[1]:
            best_match = (name, similarity)
    return best_match

# --- Xây dựng database đặc trưng khuôn mặt ---
def build_feature_database(model, tracker, data_folder='data'):
    features_db = {}
    
    for person_name in os.listdir(data_folder):
        person_path = os.path.join(data_folder, person_name)
        if not os.path.isdir(person_path):
            continue

        embeddings = []
        
        for file in os.listdir(person_path):
            if not file.lower().endswith((".jpg", ".jpeg", ".png")):
                continue
                
            img_path = os.path.join(person_path, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            
            # Phát hiện khuôn mặt trong ảnh
            results = model(img)[0]
            faces = []
            
            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                if cls == 0 and conf > 0.5:  # Class 0 là khuôn mặt và confidence > 50%
                    faces.append((x1, y1, x2, y2))
            
            if not faces:
                continue  # Bỏ qua nếu không phát hiện khuôn mặt
                
            try:
                # Lấy khuôn mặt đầu tiên
                x1, y1, x2, y2 = faces[0]
                face_crop = img[y1:y2, x1:x2]
                
                # Tiền xử lý và trích xuất đặc trưng
                processed_face = preprocess_face(face_crop)
                feature = tracker.embedder.embed(processed_face)
                embeddings.append(feature)
            except Exception as e:
                print(f"❌ Lỗi xử lý ảnh {file}: {str(e)}")
                continue
        
        if embeddings:
            avg_feature = np.mean(embeddings, axis=0)
            features_db[person_name] = avg_feature
            print(f"✅ Đã thêm {person_name} vào database")
    
    print(f"\n🎉 Hoàn thành! Tổng số người trong database: {len(features_db)}")
    return features_db

# --- Hàm chính ---
def main():
    # --- Khởi tạo YOLO model ---
    model = YOLO("model.pt")  # Model phát hiện khuôn mặt
    
    # --- Khởi tạo DeepSort tracker ---
    tracker = DeepSort(
        max_age=30,
        n_init=3,
        nms_max_overlap=1.0,
        max_cosine_distance=0.4,
        nn_budget=None,
        embedder="mobilenet",
        half=True,
        bgr=True,
        embedder_gpu=True,
    )
    
    # --- Xây dựng database đặc trưng ---
    print("\n🔍 Đang xây dựng database đặc trưng khuôn mặt...")
    features_db = build_feature_database(model, tracker, "data")
    
    # --- Mở camera ---
    cap = cv2.VideoCapture(0)
    cap.set(3, 1280)  # Chiều rộng
    cap.set(4, 720)   # Chiều cao
    
    # Lưu thông tin nhận diện cho mỗi track_id
    track_id_to_name = {}
    
    print("\n🚀 Bắt đầu nhận diện khuôn mặt real-time...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Phát hiện khuôn mặt với YOLO
        results = model(frame)[0]
        detections = []
        
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            if cls == 0 and conf > 0.5:  # Chỉ lấy khuôn mặt với confidence > 50%
                detections.append(([x1, y1, x2-x1, y2-y1], conf, 'face'))
        
        # Cập nhật tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        
        for track in tracks:
            if not track.is_confirmed():
                continue
            
            track_id = track.track_id
            ltrb = track.to_ltrb()
            x1, y1, x2, y2 = map(int, ltrb)
            
            # Nếu track_id chưa được nhận diện
            if track_id not in track_id_to_name:
                try:
                    face_crop = frame[y1:y2, x1:x2]
                    processed_face = preprocess_face(face_crop)
                    feature = tracker.embedder.embed(processed_face)
                    name, score = recognize_face(feature, features_db)
                    track_id_to_name[track_id] = (name, score)
                    print(f"👤 Nhận diện: ID {track_id} - {name} (Độ chính xác: {score:.2f})")
                except Exception as e:
                    print(f"⚠️ Lỗi nhận diện ID {track_id}: {str(e)}")
                    track_id_to_name[track_id] = ("Error", 0)
            
            # Hiển thị thông tin
            name, score = track_id_to_name.get(track_id, ("Unknown", 0))
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            
            # Vẽ bounding box và label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.rectangle(frame, (x1, y1-35), (x1+200, y1), color, -1)
            cv2.putText(frame, f"ID:{track_id} {name}", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            if name != "Unknown":
                cv2.putText(frame, f"Score: {score:.2f}", (x1, y1-30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Hiển thị FPS
        fps = cap.get(cv2.CAP_PROP_FPS)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.imshow("Face Recognition System", frame)
        
        # Thoát khi nhấn 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Giải phóng tài nguyên
    cap.release()
    cv2.destroyAllWindows()
    print("\n🛑 Đã dừng hệ thống nhận diện")

if __name__ == "__main__":
    main()

In [ ]:
import cv2
import numpy as np
import threading
import socket
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import struct

# --- Load YOLO ---
model = YOLO("model.pt")

# --- DeepSORT ---
# tracker = DeepSort(max_age=50, max_iou_distance=0.4, nn_budget=100, embedder='mobilenet', n_init=3)
tracker = DeepSort(
      max_age=30,
        n_init=3,
        nms_max_overlap=1.0,
        max_cosine_distance=0.4,
        nn_budget=None,
        embedder="mobilenet",
        half=True,
        bgr=True,
        embedder_gpu=True,
)

# --- Camera ---
cap = cv2.VideoCapture(1)
cap.set(3, 1280)
cap.set(4, 720)

# --- Socket TCP ---
HOST = '192.168.1.241'
PORT = 65432
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    client_socket.connect((HOST, PORT))
except ConnectionRefusedError:
    print("❌ Không kết nối được đến Slave.")
    exit()

def send_feature_data(feature: np.ndarray):
    try:
        serialized = feature.astype(np.float32).tobytes()
        length = struct.pack(">I", len(serialized))  # Gửi độ dài 4 byte
        client_socket.sendall(length + serialized)
        print(f"📤 Gửi đặc trưng dài {len(serialized)} byte")
    except Exception as e:
        print(f"❌ Gửi thất bại: {e}")

# --- Shared ---
detections = []
latest_frame = None
selected_id = None
yolo_lock = threading.Lock()

# --- YOLO Thread ---
def yolo_thread():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame = latest_frame.copy()
            results = model(frame)
            dets = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                if cls == 0:
                    dets.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))
            detections = dets

threading.Thread(target=yolo_thread, daemon=True).start()

# --- Mouse Callback ---
def select_object(event, x, y, flags, param):
    global selected_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for track in param:
            if track.is_confirmed():
                x1, y1, x2, y2 = map(int, track.to_ltrb())
                if x1 <= x <= x2 and y1 <= y <= y2:
                    selected_id = track.track_id
                    if hasattr(track, "features") and track.features:
                        feature = track.features[-1]
                        send_feature_data(feature)
                        print(f"🔍 Chọn và gửi ID: {selected_id}")
                    break

cv2.namedWindow("Master")
cv2.setMouseCallback("Master", select_object, param=[])

# --- Main Loop ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    with yolo_lock:
        latest_frame = frame.copy()

    tracks = tracker.update_tracks(detections, frame=frame)
    cv2.setMouseCallback("Master", select_object, param=tracks)

    for track in tracks:
        if track.is_confirmed():
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            color = (0, 255, 0) if track.track_id == selected_id else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID {track.track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Master", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
client_socket.close()
